<h2> Step 1: Add new rule to Metadata_DB.Audit.Rules table </h2>

In [ ]:
/****** CREATE & INSERT NEW RULE ******/
DECLARE @ruleOwner nvarchar(50) = 'IR' --> insert owner name here
DECLARE @ruleDescription nvarchar(150) = 'dimAcademicInstitution: SUDS_Insitution_Code, SUDS_Institution_State, BOG_Institution_Code, Florida_Institution_Code IS NOT NULL' --> insert rule name here
DECLARE @isEnabled int = 1 --> enable the rule (set as 0 to disable)

SET IDENTITY_INSERT Metadata_DB.Audit.Rules ON
INSERT INTO Metadata_DB.Audit.Rules 
([Id],[owner],ruleId,[description],isEnabled)
values (    
    (SELECT MAX(Id)+1 FROM Metadata_DB.Audit.Rules) --> set this to: max(primary key of Rules table)+1 
    , @ruleOwner 
    , (SELECT MAX(ruleId)+1 FROM Metadata_DB.Audit.Rules where owner=@ruleOwner) --> is set to: max(owner-specific ruleId)+1
    , @ruleDescription
    , @isEnabled
)
SET IDENTITY_INSERT Metadata_DB.Audit.Rules OFF

<h2> Step 2: Add record of table information to package table map </h2>

In [ ]:
/* insert new record into PackageTableMap*/
DECLARE @ID int = 281
DECLARE @packageName varchar(100) = 'Dim Update LocalDims'
DECLARE @dbName varchar(100) = 'Student_DM'
DECLARE @schemaName varchar(100) = 'dbo'
DECLARE @tableName varchar(100) = 'dimAcademicInstitution'
DECLARE @hasDates SMALLINT = 0
DECLARE @hasRules SMALLINT = 1
DECLARE @rulesOwner varchar(100) = 'IR'
SET IDENTITY_INSERT Metadata_DB.Audit.PackageTableMap ON
INSERT INTO Metadata_DB.Audit.PackageTableMap(ID, packageName, dbName, schemaName, tableName, hasDates, hasRules, rulesOwner)
VALUES(@ID, @packageName, @dbName, @schemaName, @tableName, @hasDates, @hasRules, @rulesOwner)
--VALUES (281,'Dim Update LocalDims','Student_DM','dbo','dimAcademicInstitution',0,1,'IR')
SET IDENTITY_INSERT Metadata_DB.Audit.PackageTableMap OFF

<h2>Step 3: Insert custom validation query into RunRules</h2>

This is the raw if-block that was nested inside of the containing @owner='IR' if-block:

In [ ]:
IF(@tableName = 'dimAcademicInstitution')
			BEGIN
            print('dimAcademicInstitution if-block triggered...') --> 4 DEBUG
			/* 	
				RULE: SUDS_Institution_Code IS NOT NULL 
					- the following validation query returns records where SUDS_Institution_Code IS NULL
					- rule is enforced on Student_DM.dbo.dimAcademicInstitution
			*/
			SELECT @SQL +=N'
				SELECT r.ruleid
					, owner, '''
					+@package+N'''
					, '''+@tableName+N'''
					, ''SUDS_Institution_Code''
					, s.SUDS_Institution_Code
					, s.Academic_Institution_SK
					, rowdata
				FROM '+@source+N' s
				cross apply (Select * from '+@source+N' b WHERE b.Academic_Institution_SK = s.Academic_Institution_SK for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
				join Metadata_DB.audit.rules_CLONE r on 3 = r.ruleid and r.isEnabled = 1 and r.owner = ''IR'' 
				WHERE s.SUDS_Institution_Code IS NULL AND s.SUDS_Institution_State IS NULL AND BOG_Institution_Code IS NULL AND Florida_Public_Inst_Code IS NULL' 
			END
		END 

This is a version of the procedure where the above rule is implemented. Note that originally there were no control flow structures to differentiate between tables contained within the @owner = 'IR' if-block given that the only IR-owned table with rules associagted with it was the MergedBCCSurvey table. I have since added the necessary control flow strutures to do so and have also found that the validation queries pertaining to MergedBCCSurvey may be bugged; From what I can tell they pull from column names that do not exist within their names sources. Please review and advise.

In [6]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
ALTER PROCEDURE [Audit].[RunRules] @owner nvarchar(50), @source nvarchar(50), @package nvarchar(50), @tableName nvarchar(50)
AS
BEGIN
	DECLARE @sql nvarchar(MAX) = N'', @dbSchema varchar(50) = REPLACE(@source,@tableName,'')
    print('RunRules iteration: '+' owner: '+@owner+'; source: '+@source+'; package: '+@package+'; table: '+@tableName)
	SELECT @SQL+= N'INSERT INTO [Audit].[RulesViolations]
           ([RuleId]
		   ,[Owner]
           ,[packageName]
           ,[sourceTable]
           ,[columnName]
           ,[columnValue]
           ,[recordKey]
           ,[rowdata])'

     If(@owner = 'HR') -- Human Resources rules
		BEGIN
			IF (@tableName = 'Position')
			BEGIN
				SELECT @SQL+=N'
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''PERRANK_Rank_Code'',PERRANK_Rank_Code,s.id,rowdata
					   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.id = s.id for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
					   join audit.rules r on 1 = r.ruleid and r.isEnabled = 1 and r.owner = ''HR''
					   where NBBPOSN_Table = ''22'' and PERRANK_Rank_Code is null
					   and s.Snapshot_Date = (SELECT MAX(snapshot_date) from '+@source+N')'
				SELECT @SQL+=N'		
					UNION
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''PERRANK_Rank_Code'',PERRANK_Rank_Code,s.id,rowdata
					   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.id = s.id for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
					   join audit.rules r on 2 = r.ruleid and r.isEnabled = 1 and r.owner = ''HR''
					   where NBBPOSN_Table = ''22'' and NBBPOSN_Pcls_Code in (''9001'',''9002'',''9003'',''9004'') and PERRANK_Rank_Code<>SUBSTRING(NBBPOSN_Pcls_Code,3,2)
					   and s.Snapshot_Date = (SELECT MAX(snapshot_date) from '+@source+N')'
				SELECT @SQL+=N'	
					UNION
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''PERAPPT_Tenure_Code'',PERAPPT_Tenure_Code,s.id,rowdata
					   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.id = s.id for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
					   join audit.rules r on 3 = r.ruleid and r.isEnabled = 1 and r.owner = ''HR''
					   where NBBPOSN_Table = ''22'' and PERAPPT_Tenure_Code is null
					   and s.Snapshot_Date = (SELECT MAX(snapshot_date) from '+@source+N')'
				SELECT @SQL+=N'	
					UNION
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''NBBPOSN_Jbln_Code'',NBBPOSN_Jbln_Code,s.id,rowdata
					   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.id = s.id for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
					   join audit.rules r on 4 =r.ruleid and r.isEnabled = 1 and r.owner = ''HR''
					   where NBBPOSN_Table IN (''22'',''06'') and COALESCE(NBBPOSN_Jbln_Code,'''')<>COALESCE(NBRJOBS_JBLN_CODE,'''')
					   and s.Snapshot_Date = (SELECT MAX(snapshot_date) from '+@source+N')'
				SELECT @SQL+=N'	
					UNION
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''NBBPOSN_Jbln_Code'',NBBPOSN_Jbln_Code,s.id,rowdata
					   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.id = s.id for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
					   join audit.rules r on 5 = r.ruleid and r.isEnabled = 1 and r.owner = ''HR''
					   where NBBPOSN_Table NOT IN (''22'',''06'') and COALESCE(NBBPOSN_Jbln_Code,'''')<>COALESCE(NBRJOBS_JBLN_CODE,'''')
					   and s.Snapshot_Date = (SELECT MAX(snapshot_date) from '+@source+N')'
				SELECT @SQL+=N'	
					UNION
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''SPBPERS_Citz_Code'',SPBPERS_Citz_Code,s.id,rowdata
					   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.id = s.id for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
					   join audit.rules r on 6 = r.ruleid and r.isEnabled = 1 and r.owner = ''HR''
					   where SPBPERS_Citz_Code is null
					   and s.Snapshot_Date = (SELECT MAX(snapshot_date) from '+@source+N')'
				SELECT @SQL+=N'	
					UNION
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''NTRPCLS_Eskl_Code'',NTRPCLS_Eskl_Code,s.id,rowdata
					   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.id = s.id for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
					   join audit.rules r on 7 = r.ruleid and r.isEnabled = 1 and r.owner = ''HR''
					   where NTRPCLS_Eskl_Code is null
					   and s.Snapshot_Date = (SELECT MAX(snapshot_date) from '+@source+N')'
				SELECT @SQL+=N'	
					UNION
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''NBRJOBS_FTE'',CAST(NBRJOBS_FTE as varchar(20)),s.id,rowdata
					   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.id = s.id for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
					   join audit.rules r on 8 = r.ruleid and r.isEnabled = 1 and r.owner = ''HR''
					   where NBRJOBS_FTE > 1
					   and s.Snapshot_Date = (SELECT MAX(snapshot_date) from '+@source+N')'
				SELECT @SQL+=N'	
					UNION
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''NBRJOBS_FTE'',CAST(NBRJOBS_FTE as varchar(20)),s.id,rowdata
					   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.id = s.id for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
					   join audit.rules r on 9 =r.ruleid and r.isEnabled = 1 and r.owner = ''HR''
					   where NBRJOBS_FTE = 0
					   and s.Snapshot_Date = (SELECT MAX(snapshot_date) from '+@source+N')'
				SELECT @SQL+=N'	
					UNION
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''PPRCERT_CERT_DATE_DRPE'',CAST(PPRCERT_CERT_DATE_DRPE as varchar(20)),s.id,rowdata
					   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.id = s.id for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
					   join audit.rules r on 10 = r.ruleid and r.isEnabled = 1 and r.owner = ''HR''
					   where PPRCERT_CERT_DATE_DRPE is NULL and PPRCERT_CERT_DATE_DRPT is NOT NULL
					   and s.Snapshot_Date = (SELECT MAX(snapshot_date) from '+@source+N')'
				SELECT @SQL+=N'					   			
					UNION
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''PPRCERT_CERT_DATE_DRPT'',CAST(PPRCERT_CERT_DATE_DRPT as varchar(20)),s.id,rowdata
					   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.id = s.id for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
					   join audit.rules r on 11 = r.ruleid and r.isEnabled = 1 and r.owner = ''HR''
					   where PPRCERT_CERT_DATE_DRPT is NULL and PPRCERT_CERT_DATE_DRPE is NOT NULL
					   and s.Snapshot_Date = (SELECT MAX(snapshot_date) from '+@source+N')'
				SELECT @SQL+=N'	
					UNION
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''PPRCERT_CERT_DATE_DRPE'',CAST(PPRCERT_CERT_DATE_DRPE as varchar(20)),s.id,rowdata
					   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.id = s.id for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
					   join audit.rules r on 12 = r.ruleid and r.isEnabled = 1 and r.owner = ''HR''
					   where PPRCERT_CERT_DATE_DRPE is NOT NULL and PPRCERT_CERT_DATE_DRPE > CURRENT_TIMESTAMP
					   and s.Snapshot_Date = (SELECT MAX(snapshot_date) from '+@source+N')'
				SELECT @SQL+=N'	
					UNION
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''PPRCERT_CERT_DATE_DRPT'',CAST(PPRCERT_CERT_DATE_DRPT as varchar(20)),s.id,rowdata
					   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.id = s.id for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
					   join audit.rules r on 13 = r.ruleid and r.isEnabled = 1 and r.owner = ''HR''
					   where PPRCERT_CERT_DATE_DRPT is NOT NULL and PPRCERT_CERT_DATE_DRPT < PPRCERT_CERT_DATE_DRPE
					   and s.Snapshot_Date = (SELECT MAX(snapshot_date) from '+@source+N')'
				SELECT @SQL+=N'	
					UNION
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''dateDiff(YEAR,PPRCERT_CERT_DATE_DRPE, PPRCERT_CERT_DATE_DRPT)'',CAST(dateDiff(YEAR,PPRCERT_CERT_DATE_DRPE, PPRCERT_CERT_DATE_DRPT) as varchar(20)),s.id,rowdata
					   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.id = s.id for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
					   join audit.rules r on 14 = r.ruleid and r.owner = ''HR''
					   where PPRCERT_CERT_DATE_DRPE is NOT NULL and PPRCERT_CERT_DATE_DRPT is NOT NULL and dateDiff(YEAR,PPRCERT_CERT_DATE_DRPE, PPRCERT_CERT_DATE_DRPT) > 5 
					   and s.Snapshot_Date = (SELECT MAX(snapshot_date) from '+@source+N')'
				SELECT @SQL+=N'	
					UNION
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''GORPRAC_Race_Codes'',GORPRAC_Race_Codes,s.id,rowdata
					   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.id = s.id for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
					   join audit.rules r on 15 = r.ruleid and r.isEnabled = 1 and r.owner = ''HR''
					   where SPBPERS_ETHN_CDE = ''X'' AND GORPRAC_Race_Codes is NULL 
					   and s.Snapshot_Date = (SELECT MAX(snapshot_date) from '+@source+N')'
			END

			IF(@tableName = 'Employee')
			BEGIN
				SELECT @SQL+=N'
					SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''PEBEMPL_empl_status'',PEBEMPL_empl_status,s.employee_Pk,rowdata
						From '+@source+N' s cross apply (Select * from '+@source+N' p where p.employee_PK = s.employee_Pk for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
						join audit.rules r on 16 = r.ruleid and r.isEnabled = 1 and r.owner = ''HR''
						join (SELECT nbrbjob_pidm 
							FROM '+@dbSchema+N'AssignmentsB 
							WHERE (NBRBJOB_END_DATE IS NULL OR nbrbjob_end_date >=GETDATE())
							GROUP BY NBRBJOB_PIDM, NBRBJOB_END_DATE) a on s.PEBEMPL_PIDM = a.nbrbjob_pidm
						WHERE PEBEMPL_empl_status = ''T'''
			END
		END

     If(@owner = 'PF') -- Physical Facilities rules
		BEGIN
			SELECT @SQL+=N'
				SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',CONCAT(''before_scn'','' : '',''after_scn''),CONCAT(CAST([before_scn] as varchar(20)),'' : '',CAST([after_scn] as varchar(20))),s.tablename,rowdata
				   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.tablename = s.tablename for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
				   join audit.rules r on 1 = r.ruleid and r.isEnabled = 1 and r.owner = ''PF''
				   where ([before_checksum] = [after_checksum] AND [before_scn] <> [after_scn])'
			SELECT @SQL+=N'
				UNION
				SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',CONCAT(''before_checksum'','' : '',''after_checksum''),CONCAT(CAST([before_checksum] as varchar(20)),'' : '',CAST([after_checksum] as varchar(20))),s.tablename,rowdata
				   From '+@source+N' s cross apply (Select * from '+@source+N' p where p.tablename = s.tablename for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
				   join audit.rules r on 2 = r.ruleid and r.isEnabled = 1 and r.owner = ''PF''
				   where ([before_scn] = [after_scn] AND [before_checksum] <> [after_checksum])'
		END 

     If(@owner = 'IR') -- Institutional Research rules
		BEGIN
		/* THIS ENTIRE BLOCK IS BUGGED I BELIEVE AS THERE ARE REFERENCES TO VALUES THAT NO LONGER EXISTS AT THEIR SPECIFIED SOUCES */
		If(@tableName = 'MergedBCSSESurvey')
			BEGIN
			SELECT @SQL+=N'
				SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''StudentID'',s.StudentId,s.surveyId,rowdata
				From '+@source+N' s left join dimensiondatabase.dbo.dimStudent ds on ds.UNFID = s.StudentID left join
				(
				Select surveyID, StudentID, LastName, FirstName ,ds.UNFID, ds.Last_Name, ds.First_Name, ds.Gender, IR_ODS.Staging.stringDistancePercentage(studentID, UNFID) distancePct
				from (
				SELECT  bs.surveyID, StudentID, LastName, FirstName, Case bs.Gender_Desc WHEN ''Man'' THEN ''M'' WHEN ''Woman'' THEN ''F'' ELSE ''N'' END Gender
				FROM '+@source+N' bs left join dimensiondatabase.dbo.dimStudent ds on ds.UNFID = bs.StudentID
				WHERE unfid is null
				) n join dimensiondatabase.dbo.dimStudent ds on ds.Last_Name = n.LastName and ds.First_Name = n.FirstName and ds.Gender = n.Gender
				) rc on rc.StudentID = s.StudentID and distancePct > 75
				cross apply (Select * from '+@source+N' p where p.SurveyID = s.SurveyID for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
				join audit.rules r on 1 = r.ruleid and r.isEnabled = 1 and r.owner = ''IR'' 
				WHERE ds.unfid is null and rc.StudentID is null'
			SELECT @SQL+=N'
				UNION
				SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',CONCAT(''StudentID'','' : '',''UNFID''),CONCAT(s.StudentId,'' : '',rc.UNFID),s.surveyId,rowdata
				From '+@source+N' s left join dimensiondatabase.dbo.dimStudent ds on ds.UNFID = s.StudentID left join 
				(
				Select surveyID, StudentID, LastName, FirstName ,ds.UNFID, ds.Last_Name, ds.First_Name, ds.Gender, IR_ODS.staging.stringDistancePercentage(studentID, UNFID) distancePct      
				from (
				SELECT  bs.surveyID, StudentID, LastName, FirstName, Case bs.Gender_Desc WHEN ''Man'' THEN ''M'' WHEN ''Woman'' THEN ''F'' ELSE ''N'' END Gender        
				FROM '+@source+N' bs left join dimensiondatabase.dbo.dimStudent ds on ds.UNFID = bs.StudentID        
				WHERE unfid is null
				) n join dimensiondatabase.dbo.dimStudent ds on ds.Last_Name = n.LastName and ds.First_Name = n.FirstName and ds.Gender = n.Gender
				) rc on rc.StudentID = s.StudentID
				cross apply (Select * from '+@source+N' p where p.SurveyID = s.SurveyID for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
				join audit.rules r on 2 = r.ruleid and r.isEnabled = 1 and r.owner = ''IR''
				WHERE distancePct > 75'
		END

        /* RULE ENFORCEMENT BLOCK: dimAcademicInstitution */
		IF(@tableName = 'dimAcademicInstitution')
			BEGIN
			/* 	
				RULE: SUDS_Institution_Code IS NULL AND SUDS_Institution_State IS NULL AND BOG_Institution_Code IS NULL AND Florida_Public_Inst_Code IS NULL
					- the following validation query returns records where the above rule is violated
					- rule is enforced on Student_DM.dbo.dimAcademicInstitution
			*/
			SELECT @SQL +=N'
				SELECT r.ruleid
					, owner, '''
					+@package+N'''
					, '''+@tableName+N'''
					, ''SUDS_Institution_Code''
					, s.SUDS_Institution_Code
					, s.Academic_Institution_SK
					, rowdata
				FROM '+@source+N' s
				cross apply (Select * from '+@source+N' b WHERE b.Academic_Institution_SK = s.Academic_Institution_SK for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
				join Metadata_DB.audit.rules_CLONE r on 3 = r.ruleid and r.isEnabled = 1 and r.owner = ''IR'' 
				WHERE s.SUDS_Institution_Code IS NULL AND s.SUDS_Institution_State IS NULL AND BOG_Institution_Code IS NULL AND Florida_Public_Inst_Code IS NULL' 
			END
		END 

     If(@owner = 'IDMS') -- IDMS
		BEGIN
			SELECT @SQL+=N'
				SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''FNAME'',FNAME,s.IDCardSK,rowdata
				From '+@source+N' s cross apply (Select * from '+@source+N' p where p.IDCardSK = s.IDCardSK for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
				join audit.rules r on 1 = r.ruleid and r.isEnabled = 1 and r.owner = ''IDMS''
				where FNAME like ''N[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]'' and record_end_date is null'
			SELECT @SQL+=N'
				UNION
				SELECT r.ruleid,owner,'''+@package+N''', '''+@tableName+N''',''FNAME'',FNAME,s.IDCardSK,rowdata
				From '+@source+N' s cross apply (Select * from '+@source+N' p where p.IDCardSK = s.IDCardSK for json AUTO, INCLUDE_NULL_VALUES) j (rowdata)
				join audit.rules r on 2 = r.ruleid and r.isEnabled = 1 and r.owner = ''IDMS''
				where LNAME like ''N[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]'' and record_end_date is null'
		END
	SELECT @sql
    PRINT('RunRules dynamic sql string for '+@source+': '+@sql)
	exec sp_ExecuteSQL @sql --> ON/OFF switch
END
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.006